In [1]:
import kagglehub
import shutil
import os
import cv2
import json
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Descarga el dataset si no existe en la carpeta data
dataset_final_path = "data/EuroSAT/"

if os.path.exists(dataset_final_path):
    print("El dataset ya está descargado. No es necesario volver a descargarlo")
else:
    print("Descargando el dataset...")
    download_path = kagglehub.dataset_download("apollo2506/eurosat-dataset")
    original_path = os.path.join(download_path, "EuroSAT")

    # Verifica que la carpeta EuroSAT existe en el dataset descargado
    if os.path.exists(original_path):
        # Mueve la carpeta EuroSAT a data
        shutil.move(original_path, dataset_final_path)
        print(f"Dataset movido a: {dataset_final_path}")
    else:
        print("Error: No se encontró la carpeta EuroSAT dentro del dataset")

El dataset ya está descargado. No es necesario volver a descargarlo


In [3]:
# Mapeo de etiquetas desde el fichero label_map.json
label_map_path = os.path.join(dataset_final_path, "label_map.json")

with open(label_map_path, "r") as file:
    label_map = json.load(file)

print("Etiquetas:", label_map)

Etiquetas: {'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}


In [4]:
# Extracción de características Haralick de las imágenes

# Ref. https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycomatrix
# Ref. https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops
# Ref. https://medium.com/top-python-libraries/12-examples-of-image-texture-analysis-in-python-6cf7c179ada7
# Ref. https://medium.com/@girishajmera/feature-extraction-of-images-using-glcm-gray-level-cooccurrence-matrix-e4bda8729498

# Parámetros para calcular la matriz de co-ocurrencia de niveles de gris (GLCM)
distances = [1]  # Distancia de 1 píxel entre los valores a comparar
angles = [0]  # Ángulo de 0 grados (horizontal) para evaluar la textura

X, y = [], []  # Listas para almacenar las características 'X' y etiquetas 'y'

# Recorre todas las carpetas dentro del dataset
for class_name, label in label_map.items():

    class_path = os.path.join(dataset_final_path, class_name)

    if not os.path.isdir(class_path):
        continue  # Si no es una carpeta válida se ignora

    print(f"Procesando: {class_name}")

    # Recorre todas las imágenes dentro de la carpeta actual
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        # Carga la imagen en escala de grises
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Si la imagen no se puede leer se ignora
        if img is None:
            continue

        # Calcula la matriz de co-ocurrencia de niveles de gris (GLCM)
        glcm = graycomatrix(img, distances, angles, symmetric=True, normed=True)

        # Extracción de características de Haralick
        contrast = graycoprops(glcm, 'contrast')[0, 0]  # Mide la diferencia entre valores de píxeles vecinos
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]  # Diferencia media entre píxeles
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]  # Valores similares tienen homogeneidad alta
        energy = graycoprops(glcm, 'energy')[0, 0]  # Mide cuánta uniformidad hay en la imagen
        correlation = graycoprops(glcm, 'correlation')[0, 0]  # Relación lineal entre los píxeles
        asm = graycoprops(glcm, 'ASM')[0, 0]  # Medida de uniformidad basada en probabilidad

        # Guarda el vector de características en el array 'X'
        features = [contrast, dissimilarity, homogeneity, energy, correlation, asm]
        X.append(features)

        # Guardamos la etiqueta correspondiente en la lista y
        y.append(label)

print("Extracción de features completada")

Procesando: AnnualCrop
Procesando: Forest
Procesando: HerbaceousVegetation
Procesando: Highway
Procesando: Industrial
Procesando: Pasture
Procesando: PermanentCrop
Procesando: Residential
Procesando: River
Procesando: SeaLake
Extracción de features completada


In [5]:
# Convierte los datos en arrays
X = np.array(X)
y = np.array(y)

# División de los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

Tamaño del conjunto de entrenamiento: (21600, 6)
Tamaño del conjunto de prueba: (5400, 6)


In [6]:
# Entrenamiento del modelo con SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Evaluación del modelo
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.68
